In [ ]:
!ls
%cd ../..
!ls
# Now you should see the characters-and-dialouges-association-in-comics directory

In [ ]:
import os
import subprocess
from pathlib import Path

END_WITH_LOCAL = 'characters-and-dialouges-association-in-comics'

os.environ['PATH'] = f"/root/.cargo/bin:{os.environ['PATH']}"

BASE_DIR = os.getcwd()
print(f"BASE_DIR: {BASE_DIR}")

# Simple validation
if not (BASE_DIR.endswith('/content') or BASE_DIR.endswith(END_WITH_LOCAL)):
    raise ValueError(f"Expected to be in .../{END_WITH_LOCAL} or .../content directory, but got: {BASE_DIR}")

In [ ]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")


In [ ]:
import os
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer

# --- 1. Re-register the datasets (important if in a new script/session) ---
base_path = os.path.join(BASE_DIR, "data", "Manga109_released_2023_12_07")  # Adjusted to the new dataset directory
images_dir = os.path.join(base_path, "images")
train_json_path = os.path.join(base_path, "train.json")
val_json_path = os.path.join(base_path, "val.json")

# Register the datasets
register_coco_instances("manga109_train", {}, train_json_path, images_dir)
register_coco_instances("manga109_val", {}, val_json_path, images_dir)


# --- 2. Setup Configuration ---
cfg = get_cfg()
# Use the same config file as for training

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

# cfg.MODEL.WEIGHTS = os.path.join(BASE_DIR, "models/bubble-detection/detectron2/model_final_280758.pkl")
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")


# cfg.MODEL.DEVICE = "cpu"

# Point to the model weights you just trained
cfg.MODEL.WEIGHTS = os.path.join("./output", "model_final.pth")

# Set the number of classes (must match your training config)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # body, frame

# Set the test dataset
cfg.DATASETS.TEST = ("manga109_val",)

# Set a confidence threshold for this evaluation
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # You can adjust this threshold


# --- 3. Build the Model and Run Evaluation ---

# Build the model from the config
model = build_model(cfg)
# Load the weights
DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)

# Create an evaluator for the validation dataset
# It will save detailed results in the "evaluation" subfolder of your output directory
evaluator = COCOEvaluator("manga109_val", output_dir="./output/evaluation")

# Create a data loader for the validation set
val_loader = build_detection_test_loader(cfg, "manga109_val")

# Use the official `inference_on_dataset` tool
# This will run the model on all images in the val_loader and feed the results to the evaluator
print("Running evaluation...")
results = inference_on_dataset(model, val_loader, evaluator)

# The results dictionary will also be printed, but the table above is the most readable
print("\nEvaluation results dictionary:")
print(results)